In [ ]:
%load_ext autoreload
%autoreload 2

import numpy as np
import os
import cv2
from concurrent.futures import ProcessPoolExecutor
import tqdm

import sys
sys.path.append('..')

from shared import metrics
from shared.data import KITTIData,  VisualOdometry, draw_matches, draw_keypoints
from shared import common

%matplotlib widget
import matplotlib.pyplot as plt

In [ ]:
DATASET_DIR = os.path.join('../', 'data/KITTI/dataset')
dataset = KITTIData(DATASET_DIR, sequence_id="00")
gt_poses = dataset.get_poses()
Q_left = dataset.get_left_Q_matrix()
C_left, _ = dataset.get_С_matrix()
PL, PR = dataset.get_P_matrix()

vo = VisualOdometry()

In [ ]:
def process_transform(idx):
    c_pose = gt_poses[idx]
    n_pose = gt_poses[idx+1]
    _, gt_translt = dataset._get_transform(idx)
    
#     print(f'Start idx: {idx}')
    c_l_img, c_r_img = dataset.get_images(idx)
    n_l_img, n_r_img = dataset.get_images(idx+1)

    c_depth_frame = vo.process_depth(c_l_img, c_r_img, Q_left)
    n_depth_frame = vo.process_depth(n_l_img, n_r_img, Q_left)

    c_feats, n_feats = vo.get_features(c_l_img, n_l_img)

    ### Get 3D points 
    c_pnts_3d, c_ft_idxs = vo.reproject_2d_to_3d_points(c_feats, c_depth_frame)
    n_pnts_3d, n_ft_idxs = vo.reproject_2d_to_3d_points(n_feats, n_depth_frame)

    ft_idxs = c_ft_idxs & n_ft_idxs

    c_pnts_3d = c_pnts_3d[ft_idxs]
    n_pnts_3d = n_pnts_3d[ft_idxs]
    c_feats = c_feats[ft_idxs]
    n_feats = n_feats[ft_idxs]

    ### Filter 
    if False:
        cl_idxs, _ = vo.max_clique_filter(c_pnts_3d, n_pnts_3d)
        c_pnts_3d = c_pnts_3d[cl_idxs]
        n_pnts_3d = n_pnts_3d[cl_idxs]
        c_feats = c_feats[cl_idxs]
        n_feats = n_feats[cl_idxs]
        transform = vo.get_transform(c_feats, n_feats, c_pnts_3d, n_pnts_3d, C_left, type_='PnP')
    else:
        transform = vo.get_transform(c_feats, n_feats, c_pnts_3d, n_pnts_3d, C_left, type_='PnPRansac')
    
    print(idx, transform[:3,3], gt_translt)
    
    n_pred_pose = vo.get_next_pose(transform, c_pose)
    err = np.abs(n_pred_pose-n_pose)
    if np.any(err > 0.1):
        print(f'{idx} / Too large error: {err[:3,3]}')
    if np.any(err > 1):
        raise Exception(f'>>> Failed transform: {idx}')
    
    if idx % 100 == 0:
        print(f'Transform for idx {idx} done')
    return transform

In [ ]:
poses_count = min(len(gt_poses)-1, 2000)
poses = [
    np.eye(4)
]

keyframes = None
key_idx = 0
prev_transform = None

for idx in tqdm.notebook.tqdm(range(poses_count)):
    try:
        if keyframes is None:
            c_l_img, c_r_img = dataset.get_images(idx)
            keyframes = (c_l_img, c_r_img)
            key_idx = idx
            continue
        else:
            c_l_img, c_r_img = keyframes
            n_l_img, n_r_img = dataset.get_images(idx)
            cl_feats, cr_feats, nl_feats, nr_feats = vo.get_circular_features(c_l_img, c_r_img, n_l_img, n_r_img, lk_err=20, y_err=2)  


            
        n_pnts_3d = vo.get_3d_points(nl_feats, nr_feats, PL, PR)
        c_pnts_2d = cl_feats.copy()

        ess_transform, mask = vo.get_relative_transform(cl_feats, nl_feats, C_left)
        ess_rvec = cv2.Rodrigues(ess_transform[:3,:3])[0][:,0]
        ess_tvec = ess_transform[:3,3].copy()
        
#         c_pose = gt_poses[key_idx]
#         n_pose = gt_poses[idx]
#         scale = np.linalg.norm(n_pose[:3,3]-c_pose[:3,3])
#         ess_transform[:3,3] = scale*ess_transform[:3,3]
        
        solver_data = {
#             'tvec': ess_tvec,
#             'rvec': ess_rvec
        }
#         print(f'Points {idx}/{key_idx}: {c_pnts_2d.shape[0]}')
        transform = vo.get_transform(c_pnts_2d, None, None, n_pnts_3d, C_left, type_='PnPRansac', solver_data=solver_data)
        
    #     err = np.abs(n_pred_pose-gt_poses[idx])
    #     if np.any(err > 0.1):
    #         print(f'{idx} / Too large error: {err[:3,3]}')
    #     if np.any(err > 1):
    #         raise Exception(f'>>> Failed transform: {idx}')

        inl_idxs = solver_data['inliers'][:,0]
        cl_feats_rnsc = cl_feats[inl_idxs]
        cr_feats_rnsc = cr_feats[inl_idxs]
        nl_feats_rnsc = nl_feats[inl_idxs]
        nr_feats_rnsc = nr_feats[inl_idxs]

        fix_threshold = 35
        flows = np.linalg.norm(cl_feats_rnsc-nl_feats_rnsc, axis=1)
        flow_ratio = flows[flows > fix_threshold].size/flows.size * 100

        if flow_ratio > 5:
            if prev_transform is None:
                prev_transform = transform
                is_valid = True
            else:
                is_valid = vo.check_transforms(prev_transform, transform)
            
            if is_valid:
                n_pred_pose = vo.get_next_pose(transform, poses[-1])
                # Select this as keyframe and move next
                keyframes = (n_l_img, n_r_img)
                key_idx = idx
                poses.append(n_pred_pose)
            else:
                print(f'Not valid {idx}/{key_idx}: {transform}')
    except Exception as e:
        print(f'{idx} failed: {e}')
#     else:
#         print(f'Skip {idx}')
    
# with ProcessPoolExecutor(8) as ex:
#     transforms = ex.map(process_transform, range(poses_count))
    
# for transform in transforms:
   
    
poses = np.array(poses)

# TODO - PnP without RANSAC fails

In [ ]:
plt.figure()
common.plot_trajectory(gt_poses[:poses_count])
plt.plot(poses[:,0,3],  poses[:,2,3])

metrics_ate = metrics.compute_ATE(poses, gt_poses[:len(poses)])
rpe_trns, rpe_rot = metrics.compute_RPE(poses, gt_poses[:len(poses)])
print(f"Absolute error: {metrics_ate}")
print(f"Translational error (%): {rpe_trns*100}")
print(f"Rotational error (deg/m): {rpe_rot/np.pi*180}")
print(f"Rotational error (deg/100m): {rpe_rot/np.pi*180*100}")